In [3]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [15]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2020산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [16]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [ ]:
df

In [ ]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

In [19]:
df.to_csv("2020산림복지_세이브포인트.csv", index=False, encoding='UTF-8')

In [20]:
df = pd.read_csv("2020산림복지_세이브포인트.csv", encoding='UTF-8')
df

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/naturalperso/222192091578,2020년에도 열심히 살은 나의 삶,2020.12.31.,이제는\n열심히\n부지런히\n최선을\n잘\n봉사 하는 삶을\n내려 놓아도\n잘 산다...,0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,[Sentence(text='이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려...
1,https://blog.naver.com/abc112212/222192059763,위장환경주의 ; '그린'으로 포장한 기업의 실체,2020.12.31.,"플라스틱 쓰레기 문제, 육식을 위한 가축산업으로 인한 문제, 산림 파괴, 석유 누출...",1,플라스틱 쓰레기 문제 육식을 위한 가축산업으로 인한 문제 산림 파괴 석유 누출로 인...,[Sentence(text='플라스틱 쓰레기 문제 육식을 위한 가축산업으로 인한 문...
2,https://blog.naver.com/yrangal/222191919215,[濟 經],2020.12.31.,석가 공자 예수는 성인이자 현인이기도 하다.\n이 분들은 중생을 구제하는 빼어난 지...,2,석가 공자 예수는 성인이자 현인이기도 하다 이 분들은 중생을 구제하는 빼어난 지혜가...,"[Sentence(text='석가 공자 예수는 성인이자 현인이기도 하다', star..."
3,https://blog.naver.com/jtwon21/222191901961,2020 통영거제환경연합 올해의 환경뉴스는?,2020.12.31.,2020년 통영거제환경운동연합 선정 ‘올해의 환경 뉴스’\n#통영거제환경운동연합은 ...,3,2020년 통영거제환경운동연합 선정 올해의 환경 뉴스 통영거제환경운동연합은 2020...,[Sentence(text='2020년 통영거제환경운동연합 선정 올해의 환경 뉴스 ...
4,https://blog.naver.com/inhyangin/222191824736,"전남도, 산림행정종합평가... 광양․보성·해남/세계 산림치...",2020.12.31.,"블로그\n[전남] 7,530개의 글\n[전남]\n목록열기\n전남도, 산림행정종합평가...",4,블로그 전남 7 530개의 글 전남 목록열기 전남도 산림행정종합평가 담양군 최우수 ...,[Sentence(text='블로그 전남 7 530개의 글 전남 목록열기 전남도 산...
...,...,...,...,...,...,...,...
6410,https://blog.naver.com/nature_school/221756708673,산림교육프로그램 인증 DONE!,2020.01.01.,[홍유릉 자연학교]\n홍유릉 자연학교가\n산림교육프로그램 2가지를\n동시에 인증받았...,6410,홍유릉 자연학교 홍유릉 자연학교가 산림교육프로그램 2가지를 동시에 인증받았습니다 D...,[Sentence(text='홍유릉 자연학교 홍유릉 자연학교가 산림교육프로그램 2가...
6411,https://blog.naver.com/inhyangin/221756499038,"광양시, 2020년 1월 월간 업무보고_3 [조순익 기자]",2020.01.01.,"블로그\n[공지/날씨/오늘] 3,834개의 글\n[공지/날씨/오늘]\n목록열기\n광...",6411,블로그 공지 날씨 오늘 3 834개의 글 공지 날씨 오늘 목록열기 광양시 2020년...,[Sentence(text='블로그 공지 날씨 오늘 3 834개의 글 공지 날씨 오...
6412,https://blog.naver.com/humanism1968/221756321414,(( 주간투자전략 )) 삼성전자 잠정실적(-) 예상으로 시장 및...,2020.01.01.,블로그\n공지 목록\n공지글\n글 제목 작성일\n공지 [ 주식 이야기 ] 시장이야기...,6412,블로그 공지 목록 공지글 글 제목 작성일 공지 주식 이야기 시장이야기 시장흐름 종목...,[Sentence(text='블로그 공지 목록 공지글 글 제목 작성일 공지 주식 이...
6413,https://blog.naver.com/yneconomic/221756088888,"경북도정, 땀과 열정으로 공고한 성장 기틀 마련",2020.01.01.,"지난 26일, 이철우도지사는 도청에서 기자간담회를 열고, 2019년 도정의 주요성과...",6413,지난 26일 이철우도지사는 도청에서 기자간담회를 열고 2019년 도정의 주요성과와 ...,[Sentence(text='지난 26일 이철우도지사는 도청에서 기자간담회를 열고 ...


In [8]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./산림복지/2020산림복지_본문_전처리.csv", index=False, encoding='UTF-8')

In [21]:
df2 = pd.read_csv("./산림복지/2020산림복지_본문_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [22]:
df_1 = df[:1000]
df_2 = df[1000:2000]
df_3 = df[2000:3000]
df_4 = df[3000:4000]
df_5 = df[4000:5000]
df_6 = df[5000:]

In [23]:
df_1['split_list'].index

RangeIndex(start=0, stop=1000, step=1)

In [24]:
for i in tqdm(df_1['split_list'].index):
    for j in range(len(df_1['split_list'][i])):
        split_str= {'split_str': df_1['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

[1]


KeyboardInterrupt: 

In [ ]:
for i in tqdm(df_2['split_list'].index):
    for j in range(len(df_2['split_list'][i])):
        split_str= {'split_str': df_2['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [52]:
for i in tqdm(df_3['split_list'].index):
    for j in range(len(df_3['split_list'][i])):
        split_str= {'split_str': df_3['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)

[2999]


In [ ]:
for i in tqdm(df_4['split_list'].index):
    for j in range(len(df_4['split_list'][i])):
        split_str= {'split_str': df_4['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
for i in tqdm(df_5['split_list'].index):
    for j in range(len(df_5['split_list'][i])):
        split_str= {'split_str': df_5['split_list'][i][j][0], 'org_idx': [i]}
        print(split_str['org_idx'])
        split_df= pd.DataFrame(split_str)
        df2 = pd.concat([df2,split_df])
        df2.reset_index(drop=True, inplace=True)
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

In [21]:
df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')

In [14]:
df2 # 3999까지 한거.

,split_str,org_idx
0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0
1,momo 2018 12 31 19 52 https blog naver com omj...,0
2,An enemy generally says and believes what he w...,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,0
4,Carpe diem 현재를 즐겨라,0
...,...,...
110259,김종욱 이사장은 상대적으로 산림복지서비스 기회가 적은 취약계층에게 숲을 통한 소통 ...,3998
110260,종합지 대한뉴스 등록번호 서울가361호 크리에이티브 코리아 등록번호 강서라00175...,3998
110261,Tag 저작권자 대한뉴스 무단전재 및 재배포 금지 공감 2 이 글에 공감한 블로거 ...,3998
110262,세월속에 세월속에 139개의 글 세월속에 목록열기 목민심서12편72조 0414 세월...,3999


In [ ]:
#- 패키지 임포트
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm
import re


'''맞춤법 검사 함수'''
def ko_grammar(sent):
    global correct_list ; correct_list= []
    #for i in tqdm(range(len(a))):
    for i in tqdm(range(len(sent))):
        try:                                                                               
            response = requests.post('http://164.125.7.61/speller/results', data={'text1': sent[i]})  # 맞춤법 검사 요청 (requests)    
            data = response.text.split('data = [', 1)[-1].rsplit('];', 1)[0] # 응답에서 필요한 내용 추출 (html 파싱)
            data = json.loads(data) # 파이썬 딕셔너리 형식으로 변환
            orgStr = [err['orgStr'] for err in data['errInfo']] #오류가 담긴 errinfo에서 고쳐야 되는 단어 리스트로 추출 
            correct = [err['candWord'] for err in data['errInfo']] #고쳐진 단어도 리스트로 추출
            splited = sent[i] 

            '''여러 개로 고쳐졌을 때 맨 처음 단어만 선택'''
            for j in range(len(correct)):
                if '|'in correct[j]: #여러 개로 고쳐질 경우 문자열이 '|'로 구분되어 있음
                    com = re.compile('\|')
                    many = com.search(correct[j])
                    correct[j]= correct [j][0:(many.span()[0])]
                '''틀린 문장 고치기'''   
                splited = splited.replace(orgStr[j], correct[j])

        except:      
            splited = sent[i]

        correct_list.append(splited)

# 데이터 프레임 나누기
df2_1 = df2[:5000]
df2_2 = df2[5000:10000]
df2_3 = df2[10000:15000]

df2_n = df2[:]

'''인덱스 0부터'''
df2_1.reset_index(drop=True, inplace=True)
df2_2.reset_index(drop=True, inplace=True)
df2_3.reset_index(drop=True, inplace=True)

df2_n.reset_index(drop=True, inplace=True)


'''맞춤법 검사하기''' 
ko_grammar(df2_1['split_str'])
df2_n['correct_str'] = correct_list
df2_n.to_csv('df2_n.txt', encoding='UTF-8')